In [5]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import operator


In [6]:
dataset = pd.read_json(open("train.json"))
dataset = dataset.iloc[:100,:]
print(dataset.shape)
dataset.head()

(100, 3)


,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [7]:
print(dataset.shape)
dataset = dataset.drop('id', axis = 1)
print(sum(dataset['cuisine'].isnull()))
print(sum(dataset['ingredients'].isnull()))

(100, 3)
0
0


In [8]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.countplot(y='cuisine', data=dataset)
plt.gcf().set_size_inches(8,5)
plt.title('Cuisine Distribution',size=1)
plt.xlabel('No. of Recipes')
plt.ylabel('Cuisine')

Text(0,0.5,'Cuisine')

In [9]:
cuisines = dataset['cuisine'].unique()
cuisines.shape
print("Cuisines : ", len(cuisines))


Cuisines :  19


In [10]:
mod_cuisine = []
for c in dataset['cuisine']:
  mod_cuisine.append(c.capitalize())
dataset['cuisine'] = mod_cuisine

In [11]:
def extract_unique_items(ingredients):
    unique_items = []
    for items in ingredients:
        for item in items:
            if item not in unique_items:
                unique_items.append(item)
    return unique_items
ingredients = extract_unique_items(dataset['ingredients'])
print("unique ingredients : ", len(ingredients))

unique ingredients :  477


In [12]:
for ingredient in ingredients:
    dataset[ingredient] = np.zeros(len(dataset["ingredients"]))
dataset.head()

,cuisine,ingredients,romaine lettuce,black olives,grape tomatoes,garlic,pepper,purple onion,seasoning,garbanzo beans,...,hominy,tomatillos,pepitas,catfish fillets,basmati rice,cauliflower,cumin,blood orange,low salt chicken broth,white wine vinegar
0,Greek,"[romaine lettuce, black olives, grape tomatoes...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Indian,"[water, vegetable oil, wheat, salt]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Indian,"[black pepper, shallots, cornflour, cayenne pe...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def find_item(ingredients_list, dataset):
    position = 0
    for items in ingredients_list:
        for item in items:
            if item in ingredients:
                dataset.loc[position, item] = 1
            else:
                pass
        position = position + 1
find_item(dataset["ingredients"], dataset)
dataset.head()

,cuisine,ingredients,romaine lettuce,black olives,grape tomatoes,garlic,pepper,purple onion,seasoning,garbanzo beans,...,hominy,tomatillos,pepitas,catfish fillets,basmati rice,cauliflower,cumin,blood orange,low salt chicken broth,white wine vinegar
0,Greek,"[romaine lettuce, black olives, grape tomatoes...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Indian,"[water, vegetable oil, wheat, salt]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Indian,"[black pepper, shallots, cornflour, cayenne pe...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
X = dataset[ingredients]
Y = dataset['cuisine']
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [15]:
def getHammingDistance(test_instance, train_instance):
  distance = 0
  l = len(test_instance)
  for i in range(l):
    if test_instance.iloc[i] != train_instance.iloc[i]:
      distance += 1
  return distance

In [16]:
def getKNearNeighbors(X_train, Y_train, test_instance, k):
  distances = []
  for x in range(len(X_train)):
    distance = getHammingDistance( X_train.iloc[x, :], test_instance)
    distances.append((Y_train.iloc[x], distance))
  distances.sort(key=operator.itemgetter(1))
  neighbors = []
  for x in distances[:k]:
    neighbors.append(x)
  return neighbors
    

In [17]:
neighbors = getKNearNeighbors(X_train.head(), Y_train.head(), X_test.iloc[0,:], 2)
neighbors[0][1]

13

In [18]:
def getPredictedClass(neighbors):
  classVotes = {}
  for x in range(len(neighbors)):
    predictedClass = neighbors[x][0]
    if predictedClass in classVotes:
      classVotes[predictedClass] += 1
    else:
        classVotes[predictedClass] = 1
  sortedVotes = sorted(classVotes, key=operator.itemgetter(1), reverse=True)
  return sortedVotes[0]

In [19]:
predictedClass = getPredictedClass(neighbors)
predictedClass

'Italian'

In [20]:
def getAccuracy(Y_test, predictions):
  correct = 0
  l = len(predictions)
  for i in range(l):
    if Y_test.iloc[i] == predictions[i]:
      correct += 1
  return correct / l

In [21]:
Y_train.shape

(80,)

In [22]:
predictions=[]
k = 3
l = len(X_test)
for x in range(l):
  neighbors = getKNearNeighbors(X_train, Y_train,X_test.iloc[x, :], k)
  result = getPredictedClass(neighbors)
  predictions.append(result)
print(predictions)
getAccuracy(Y_test, predictions)
  

['Southern_us', 'Italian', 'Italian', 'Southern_us', 'French', 'Italian', 'French', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Southern_us', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Italian', 'Southern_us']


0.35

In [23]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [24]:
y_pred = classifier.predict(X_test)


In [25]:
accuracy_score(Y_test, y_pred)

0.2